# YOLOv8 Export with Embeddings

Export YOLOv8n to ONNX format with multiple outputs:
- Detection output (bounding boxes, classes, confidences)
- Embedding output (backbone features for similarity search)

## Imports

In [1]:
import torch
import torch.nn as nn
from ultralytics import YOLO
import onnx
import onnxruntime as ort
import numpy as np
import polars as pl
from pathlib import Path

2026-02-14 18:32:04.257206712 [W:onnxruntime:Default, device_discovery.cc:131 GetPciBusId] Skipping pci_bus_id for PCI path at "/sys/devices/LNXSYSTM:00/LNXSYBUS:00/ACPI0004:00/MSFT1000:00/5620e0c7-8062-4dce-aeb7-520c7ef76171" because filename ""5620e0c7-8062-4dce-aeb7-520c7ef76171"" dit not match expected pattern of [0-9a-f]+:[0-9a-f]+:[0-9a-f]+[.][0-9a-f]+


## Setup

In [2]:
# Setup model directory
model_dir = Path("var/models")
model_dir.mkdir(parents=True, exist_ok=True)

print(f"Model directory: {model_dir.absolute()}")
print(f"ONNX Runtime: {ort.__version__}")

print(f"ONNX: {onnx.__version__}")
print(f"PyTorch: {torch.__version__}")



Model directory: /home/bryanc/repos/rook_lifewatch/rook_lw_model_dev/var/models
ONNX Runtime: 1.24.1
ONNX: 1.20.1
PyTorch: 2.10.0+cu128


## Load YOLOv8 Nano

In [3]:
model_path = model_dir / 'yolov8n.pt'
model = YOLO(str(model_path))
print(f"✓ YOLOv8n loaded from {model_path}")
model.info()

✓ YOLOv8n loaded from var/models/yolov8n.pt
YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(129, 3157200, 0, 8.8575488)

## Inspect Architecture

In [4]:
pt_model = model.model

# Find key architectural boundaries
# First pass: find architectural boundaries
sppf_layer_idx = None
first_upsample_idx = None

for i, module in enumerate(pt_model.model):
    layer_type = module.__class__.__name__
    
    if layer_type == "SPPF":
        sppf_layer_idx = i
    
    if layer_type == "Upsample" and first_upsample_idx is None:
        first_upsample_idx = i

# Second pass: build data with section labels
layers_data = []
for i, module in enumerate(pt_model.model):
    layer_type = module.__class__.__name__
    
    # Determine section based on discovered boundaries
    if i <= sppf_layer_idx:
        section = "Backbone"
    elif layer_type == "Detect":
        section = "Head"
    else:
        section = "Neck"
    
    layers_data.append({"Index": i, "Type": layer_type, "Section": section})

df = pl.DataFrame(layers_data)
print("\nYOLOv8n Layers:")
with pl.Config(tbl_rows=-1):  # Show all rows (-1), or use a number like 50
    print(df)

print(f"\n📍 Embedding Extraction Point:")
print(f"   Layer {sppf_layer_idx} (SPPF) = End of backbone, best for embeddings")
print(f"   - Rich semantic features (~512 channels)")
print(f"   - Global context via spatial pyramid pooling")
print(f"   - Before multi-scale fusion (neck) begins")
print(f"\n   Architecture breakdown:")
print(f"   - Backbone: Layers 0-{sppf_layer_idx}")
print(f"   - Neck: Layers {first_upsample_idx}-{len(pt_model.model)-2}")
print(f"   - Head: Layer {len(pt_model.model)-1}")


YOLOv8n Layers:
shape: (23, 3)
┌───────┬──────────┬──────────┐
│ Index ┆ Type     ┆ Section  │
│ ---   ┆ ---      ┆ ---      │
│ i64   ┆ str      ┆ str      │
╞═══════╪══════════╪══════════╡
│ 0     ┆ Conv     ┆ Backbone │
│ 1     ┆ Conv     ┆ Backbone │
│ 2     ┆ C2f      ┆ Backbone │
│ 3     ┆ Conv     ┆ Backbone │
│ 4     ┆ C2f      ┆ Backbone │
│ 5     ┆ Conv     ┆ Backbone │
│ 6     ┆ C2f      ┆ Backbone │
│ 7     ┆ Conv     ┆ Backbone │
│ 8     ┆ C2f      ┆ Backbone │
│ 9     ┆ SPPF     ┆ Backbone │
│ 10    ┆ Upsample ┆ Neck     │
│ 11    ┆ Concat   ┆ Neck     │
│ 12    ┆ C2f      ┆ Neck     │
│ 13    ┆ Upsample ┆ Neck     │
│ 14    ┆ Concat   ┆ Neck     │
│ 15    ┆ C2f      ┆ Neck     │
│ 16    ┆ Conv     ┆ Neck     │
│ 17    ┆ Concat   ┆ Neck     │
│ 18    ┆ C2f      ┆ Neck     │
│ 19    ┆ Conv     ┆ Neck     │
│ 20    ┆ Concat   ┆ Neck     │
│ 21    ┆ C2f      ┆ Neck     │
│ 22    ┆ Detect   ┆ Head     │
└───────┴──────────┴──────────┘

📍 Embedding Extraction Point:
   Layer 

## Create Multi-Output Wrapper

In [5]:
class YOLOv8WithEmbeddings(nn.Module):
    """
    Wrapper that extracts both detections and embeddings from YOLOv8.
    
    Architecture:
    - Backbone: Feature extraction, ends with SPPF layer
    - Neck: Multi-scale feature fusion (starts with Upsample)
    - Head: Detection output (Detect layer)
    
    This wrapper captures backbone features at the SPPF layer which provides:
    - High-level semantic representations (~512 channels)
    - Global spatial context via pyramid pooling
    - Pre-fusion features ideal for similarity comparison
    """
    def __init__(self, yolo_model, embedding_layer_idx=None):
        super().__init__()
        # Get the underlying DetectionModel from YOLO wrapper
        self.model = yolo_model.model
        
        # Auto-detect embedding layer if not specified
        if embedding_layer_idx is None:
            # Find SPPF layer (end of backbone)
            for i, layer in enumerate(self.model.model):
                if layer.__class__.__name__ == "SPPF":
                    embedding_layer_idx = i
                    break
            
            if embedding_layer_idx is None:
                raise RuntimeError("Could not find SPPF layer in model architecture")
        
        self.embedding_layer_idx = embedding_layer_idx
        self.embedding_layer = self.model.model[self.embedding_layer_idx]
        self._embeddings = None
        
        # Register hook to capture SPPF output
        self.embedding_layer.register_forward_hook(self._hook_fn)
        
        print(f"Extracting embeddings from layer {self.embedding_layer_idx} ({self.embedding_layer.__class__.__name__})")
    
    def _hook_fn(self, module, input, output):
        """Hook function to capture SPPF layer output"""
        # Apply global average pooling: [Batch, Channels, Height, Width] -> [Batch, Channels]
        self._embeddings = torch.nn.functional.adaptive_avg_pool2d(output, (1, 1)).squeeze(-1).squeeze(-1)
        
    def forward(self, x):
        # Reset embeddings
        self._embeddings = None
        
        # Run normal YOLOv8 forward pass (hook will capture SPPF output)
        detections = self.model(x)
        
        # YOLOv8 may return tuple/list of outputs, extract the main detection tensor
        if isinstance(detections, (list, tuple)):
            detections = detections[0]
        
        if self._embeddings is None:
            raise RuntimeError(f"Failed to capture embeddings from layer {self.embedding_layer_idx}")
        
        # detections: [1, 84, 8400] - (4 bbox coords + 80 class scores, 8400 predictions)
        # embeddings: [1, channels] - feature vector for similarity search
        return detections, self._embeddings

# Create wrapper - automatically finds SPPF layer
wrapped_model = YOLOv8WithEmbeddings(model)
wrapped_model.eval()
print("✓ Wrapper created")

Extracting embeddings from layer 9 (SPPF)
✓ Wrapper created


## Test Wrapper

In [6]:
dummy_input = torch.randn(1, 3, 640, 640)

with torch.no_grad():
    detections, embeddings = wrapped_model(dummy_input)

print(f"Detections: {detections.shape}")
print(f"Embeddings: {embeddings.shape}")
print(f"✓ {embeddings.shape[1]}-dim feature vector")

Detections: torch.Size([1, 84, 8400])
Embeddings: torch.Size([1, 256])
✓ 256-dim feature vector


## Export to ONNX

In [ ]:
output_path = model_dir / "yolov8n_with_embeddings.onnx"

torch.onnx.export(
    wrapped_model,
    dummy_input,
    output_path,
    export_params=True,
    opset_version=17,
    do_constant_folding=True,
    input_names=['images'],
    output_names=['detections', 'embeddings'],
    dynamic_axes={
        'images': {0: 'batch'},
        'detections': {0: 'batch'},
        'embeddings': {0: 'batch'}
    },
    # Keep all data in single file (no external .data file)
    dynamo=False
)

size_mb = output_path.stat().st_size / 1024 / 1024
print(f"✓ Exported: {output_path.absolute()}")
print(f"  Size: {size_mb:.2f} MB")

/tmp/ipykernel_100566/4001159241.py:3: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0214 18:32:04.586000 100566 torch/onnx/_internal/exporter/_compat.py:125] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W0214 18:32:05.012000 100566 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = 

[torch.onnx] Obtain model graph for `YOLOv8WithEmbeddings([...]` with `torch.export.export(..., strict=False)`...


/usr/lib/python3.13/contextlib.py:148: UserWarning: The tensor attribute self._embeddings was assigned during export. Such attributes must be registered as buffers using the `register_buffer` API (https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.register_buffer).
  next(self.gen)


[torch.onnx] Obtain model graph for `YOLOv8WithEmbeddings([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


/usr/lib/python3.13/copyreg.py:99: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls.__new__(cls, *args)
The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 17).
Failed to convert the model to the target version 17 using the ONNX C API. The model was not modified
Traceback (most recent call last):
  File "/home/bryanc/repos/rook_lifewatch/rook_lw_model_dev/.venv/lib/python3.13/site-packages/onnxscript/version_converter/__init__.py", line 120, in call
    converted_proto = _c_api_utils.call_onnx_api(
        func=_partial_convert_version, model=model
    )
  File "/home/bryanc/repos/rook_lifewatch/rook_lw_model_dev/.venv/lib/python3.13/site-packages/onnxscript/version_converter/_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
  File "/home/bryanc/repos/ro

[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 123 of general pattern rewrite rules.
✓ Exported: /home/bryanc/repos/rook_lifewatch/rook_lw_model_dev/var/models/yolov8n_with_embeddings.onnx
  Size: 0.49 MB


## Verify ONNX Model

In [8]:
onnx_model = onnx.load(str(output_path))
onnx.checker.check_model(onnx_model)
print("✓ ONNX model valid")

print("\nInputs:")
for inp in onnx_model.graph.input:
    print(f"  {inp.name}")

print("\nOutputs:")
for out in onnx_model.graph.output:
    print(f"  {out.name}")

✓ ONNX model valid

Inputs:
  images

Outputs:
  detections
  embeddings


## Test ONNX Runtime

In [9]:
session = ort.InferenceSession(str(output_path))
input_name = session.get_inputs()[0].name
output_names = [o.name for o in session.get_outputs()]

outputs = session.run(output_names, {input_name: dummy_input.numpy()})
det_onnx, emb_onnx = outputs

print(f"✓ ONNX Runtime inference successful")
print(f"Detections: {det_onnx.shape}")
print(f"Embeddings: {emb_onnx.shape}")

# Compare with PyTorch
with torch.no_grad():
    det_pt, emb_pt = wrapped_model(dummy_input)

det_diff = np.abs(det_onnx - det_pt.numpy()).max()
emb_diff = np.abs(emb_onnx - emb_pt.numpy()).max()

print(f"\nMax diff from PyTorch:")
print(f"  Detections: {det_diff:.6f}")
print(f"  Embeddings: {emb_diff:.6f}")

✓ ONNX Runtime inference successful
Detections: (1, 84, 8400)
Embeddings: (1, 256)

Max diff from PyTorch:
  Detections: 0.001373
  Embeddings: 0.000002


## Summary

Model exported with dual outputs:
- `detections`: [1, 84, 8400] - Standard YOLOv8 format
- `embeddings`: [1, channels] - Feature vector

Ready for use in Rust/ONNX Runtime!